# Week 3 Fuzzy function implementation

In [2]:
import os
import tarfile
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
import re
import pyranges as pr

In [4]:
from google.colab import drive
drive.mount('/content/MyDrive/')

Drive already mounted at /content/MyDrive/; to attempt to forcibly remount, call drive.mount("/content/MyDrive/", force_remount=True).


In [3]:
ls

MyDrive/  sample_data/


In [4]:
cd MyDrive/MyDrive/Z.Chen/

/content/MyDrive/MyDrive/Z.Chen


In [5]:
df = pd.read_csv('./iMARGI_extracted/GSM4006840_HUVEC_control_iMARGI.bedpe.gz', sep='\t', header=None)

In [6]:
ls

6840.png                       genes_df.csv                 Untitled0.ipynb
coexp.py                       iMARGI_extracted/            Week1.ipynb
coexp_update.py                position_info_LINC00607.png  Week2.ipynb
gencode.v38.annotation.gtf.gz  __pycache__/


In [7]:
gene_names = pd.read_csv('genes_df.csv')

In [19]:
gene_names.head(2)

,Chromosome,Start,End,gene_name
0,chr1,11868,14409,DDX11L1
1,chr1,29553,31109,MIR1302-2HG


In [8]:
genes = gene_names[['Chromosome','Start','End','gene_name']]

In [10]:
genes

,Chromosome,Start,End,gene_name
0,chr1,11868,14409,DDX11L1
1,chr1,29553,31109,MIR1302-2HG
2,chr1,30365,30503,MIR1302-2
3,chr1,52472,53312,OR4G4P
4,chr1,57597,64116,OR4G11P
...,...,...,...,...
60644,chrY,57015104,57016096,AMD1P2
60645,chrY,57165511,57165845,ELOCP24
60646,chrY,57171889,57172769,TRPC6P
60647,chrY,57201142,57203357,WASIR1


In [9]:
df.columns = [
            "RNA_chr", "RNA_start", "RNA_end",
            "DNA_chr", "DNA_start", "DNA_end",
            "name", "score", "RNA_strand", "DNA_strand"
        ]

In [23]:
if df.shape[1] == 10:

    # assign column names
    df.columns = [
        "RNA_chr", "RNA_start", "RNA_end",
        "DNA_chr", "DNA_start", "DNA_end",
        "name", "score", "RNA_strand", "DNA_strand"
    ]


    # left join RNA names with hg38 reference
    rna_annot = genes.rename(columns={
    'Chromosome':'RNA_chr',
    'Start'     :'RNA_start',
    'End'       :'RNA_end',
    'gene_name' :'RNA_gene_name'})

    df = df.merge(
        rna_annot,
        how = 'left',
        on = ['RNA_chr', 'RNA_start', 'RNA_end']
    )

    # doing the same for DNA names
    dna_annot = genes.rename(columns={
        'Chromosome':'DNA_chr',
        'Start'     :'DNA_start',
        'End'       :'DNA_end',
        'gene_name' :'DNA_gene_name'})

    df = df.merge(
        dna_annot,
        how = 'left',
        on = ['DNA_chr', 'DNA_start', 'DNA_end']
    )


In [24]:
df.head(2)

,RNA_chr,RNA_start,RNA_end,DNA_chr,DNA_start,DNA_end,name,score,RNA_strand,DNA_strand,RNA_gene_name,DNA_gene_name
0,chr4,40944863,40944903,chr4,40944853,40944901,.,.,+,-,NaN,NaN
1,chr12,53043128,53043227,chr12,53043250,53043351,.,.,+,-,NaN,NaN


In [25]:
df

,RNA_chr,RNA_start,RNA_end,DNA_chr,DNA_start,DNA_end,name,score,RNA_strand,DNA_strand,RNA_gene_name,DNA_gene_name
0,chr4,40944863,40944903,chr4,40944853,40944901,.,.,+,-,NaN,NaN
1,chr12,53043128,53043227,chr12,53043250,53043351,.,.,+,-,NaN,NaN
2,chr4,40924463,40924535,chr3,111255038,111255094,.,.,+,-,NaN,NaN
3,chr1,179100935,179101034,chr1,179100959,179101060,.,.,+,-,NaN,NaN
4,chr18,61758386,61758485,chr18,61758326,61758427,.,.,-,+,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
66258767,chr15,58646038,58646085,chr15,58646049,58646085,.,.,+,-,NaN,NaN
66258768,chr12,6058362,6058436,chr7,3789771,3789821,.,.,+,-,NaN,NaN
66258769,chr12,21646949,21647028,chr11,71348930,71348994,.,.,+,+,NaN,NaN
66258770,chr17,10505340,10505439,chr17,10505285,10505386,.,.,-,+,NaN,NaN


In [26]:
df = df[['RNA_chr', 'RNA_start', 'RNA_end', 'DNA_chr', 'DNA_start', 'DNA_end']]
df['RNA_regions'] = df['RNA_chr'] + ':' + df['RNA_start'].astype(str) + '-' + df['RNA_end'].astype(str)
df['DNA_regions'] = df['DNA_chr'] + ':' + df['DNA_start'].astype(str) + '-' + df['DNA_end'].astype(str)

/tmp/ipython-input-26-2192674025.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['RNA_regions'] = df['RNA_chr'] + ':' + df['RNA_start'].astype(str) + '-' + df['RNA_end'].astype(str)
/tmp/ipython-input-26-2192674025.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DNA_regions'] = df['DNA_chr'] + ':' + df['DNA_start'].astype(str) + '-' + df['DNA_end'].astype(str)


In [27]:
df.head(5)

,RNA_chr,RNA_start,RNA_end,DNA_chr,DNA_start,DNA_end,RNA_regions,DNA_regions
0,chr4,40944863,40944903,chr4,40944853,40944901,chr4:40944863-40944903,chr4:40944853-40944901
1,chr12,53043128,53043227,chr12,53043250,53043351,chr12:53043128-53043227,chr12:53043250-53043351
2,chr4,40924463,40924535,chr3,111255038,111255094,chr4:40924463-40924535,chr3:111255038-111255094
3,chr1,179100935,179101034,chr1,179100959,179101060,chr1:179100935-179101034,chr1:179100959-179101060
4,chr18,61758386,61758485,chr18,61758326,61758427,chr18:61758386-61758485,chr18:61758326-61758427


In [28]:
genes.head(5)

,Chromosome,Start,End,gene_name
0,chr1,11868,14409,DDX11L1
1,chr1,29553,31109,MIR1302-2HG
2,chr1,30365,30503,MIR1302-2
3,chr1,52472,53312,OR4G4P
4,chr1,57597,64116,OR4G11P


In [30]:
df = df.drop(columns=['RNA_regions', 'DNA_regions'])

In [31]:
df.head(5)

,RNA_chr,RNA_start,RNA_end,DNA_chr,DNA_start,DNA_end
0,chr4,40944863,40944903,chr4,40944853,40944901
1,chr12,53043128,53043227,chr12,53043250,53043351
2,chr4,40924463,40924535,chr3,111255038,111255094
3,chr1,179100935,179101034,chr1,179100959,179101060
4,chr18,61758386,61758485,chr18,61758326,61758427


In [32]:

# Set the fuzzy tolerance in base pairs
TOLERANCE = 1000

# Function to match a region to a gene name in gene reference
def match_gene(chrom, start, end, gene_df, tolerance=TOLERANCE):
    matched = gene_df[
        (gene_df['Chromosome'] == chrom) &
        (abs(gene_df['Start'] - start) <= tolerance) &
        (abs(gene_df['End'] - end) <= tolerance)
    ]
    if not matched.empty:
        return matched.iloc[0]['gene_name']
    else:
        return None




In [35]:
# did not work for intense CPU used (run for more than 20min and still no sign of stop)

df['RNA_gene_name'] = df.apply(
    lambda row: match_gene(row['RNA_chr'], row['RNA_start'], row['RNA_end'], genes),
    axis=1
)


KeyboardInterrupt: 

In [36]:
genes['Chromosome'].unique()

array(['chr1', 'chr2', 'chr3', 'chr4', 'chr5', 'chr6', 'chr7', 'chr8',
       'chr9', 'chr10', 'chr11', 'chr12', 'chr13', 'chr14', 'chr15',
       'chr16', 'chr17', 'chr18', 'chr19', 'chr20', 'chr21', 'chr22',
       'chrM', 'chrX', 'chrY'], dtype=object)

In [38]:
pip install intervaltree

  Preparing metadata (setup.py) ... done
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26098 sha256=bb344d33ab8861a8544ba7fef5897555cf03ad87b7803210e1eb235fcf34e4d6
  Stored in directory: /root/.cache/pip/wheels/31/d7/d9/eec6891f78cac19a693bd40ecb8365d2f4613318c145ec9816
Successfully built intervaltree


In [41]:
from intervaltree import IntervalTree

# Build tree from genes_df
tree_dict = {}

for chrom in genes['Chromosome'].unique():
    tree = IntervalTree()
    subset = genes[genes['Chromosome'] == chrom]
    for _, row in subset.iterrows():
        # Add with small buffer/tolerance if needed
        tree[row['Start']:row['End']] = row['gene_name']
    tree_dict[chrom] = tree

# Match function using tree
def fast_gene_lookup(chrom, start, end):
    tree = tree_dict.get(chrom, None)
    if not tree:
        return None
    hits = tree[start:end]
    return list(hits)[0].data if hits else None



In [1]:
# Apply faster
df['RNA_gene_name'] = df.apply(
    lambda row: fast_gene_lookup(row['RNA_chr'], row['RNA_start'], row['RNA_end']),
    axis=1
)


NameError: name 'df' is not defined

In [ ]:
# Apply faster
df['DNA_gene_name'] = df.apply(
    lambda row: fast_gene_lookup(row['DNA_chr'], row['DNA_start'], row['DNA_end']),
    axis=1
)


In [10]:
import pyranges as pr

# Create PyRanges objects
query = pr.PyRanges(df.rename(columns={
    'RNA_chr': 'Chromosome',
    'RNA_start': 'Start',
    'RNA_end': 'End'
}))

genes = pr.PyRanges(genes)

# Perform overlap
overlap = query.join(genes, how='left')

# Result contains gene_name
df['RNA_gene_name'] = overlap.df['gene_name']

KeyboardInterrupt: 

In [14]:
import pyranges as pr

# Set fuzzy tolerance
delta = 1000

# Create fuzzy RNA regions by extending Start/End
df['RNA_start_fuzzy'] = df['RNA_start'] - delta
df['RNA_end_fuzzy'] = df['RNA_end'] + delta

# Create query PyRanges
query = pr.PyRanges(df.rename(columns={
    'RNA_chr': 'Chromosome',
    'RNA_start_fuzzy': 'Start',
    'RNA_end_fuzzy': 'End'
}))

# Create gene reference PyRanges (assuming genes_df already loaded)
genes = pr.PyRanges(genes.rename(columns={
    'Chromosome': 'Chromosome',
    'Start': 'Start',
    'End': 'End',
    'gene_name': 'gene_name'
}))

In [ ]:
overlap = query.join(genes, how='left')